In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
import torch.optim as optim
from torch.jit import script, trace
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision.transforms import v2

from tqdm import tqdm

from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

c:\Users\mauma\anaconda3\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\mauma\anaconda3\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: h

In [ ]:
x_train = None
y_train = None

x_valid = None
y_valid = None

x_test = None
y_test = None

In [ ]:
train_subset = torch.utils.data.TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
val_subset = torch.utils.data.TensorDataset(torch.Tensor(x_valid), torch.Tensor(y_valid))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

bsz = 20
train_loader = torch.utils.data.DataLoader(train_subset, shuffle=True, batch_size=bsz)
val_loader = torch.utils.data.DataLoader(val_subset, shuffle=False, batch_size=bsz)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=bsz)

In [2]:
samp_time = 750
n_channels = 8

In [5]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=8, out_channels=40, kernel_size=20, stride=4)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.batchnorm1 = nn.BatchNorm1d(40)
        self.maxpool = nn.MaxPool1d(kernel_size=4, stride=4)
        self.lstm1 = nn.LSTM(input_size=45, hidden_size=30)
        self.dropout2 = nn.Dropout(p=0.5)
        self.batchnorm2 = nn.BatchNorm1d(40)
        self.lstm2 = nn.LSTM(input_size=30, hidden_size=10)
        self.dropout3 = nn.Dropout(p=0.5)
        self.batchnorm3 = nn.BatchNorm1d(40)
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(400,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.batchnorm1(x)
        x = self.maxpool(x)
        x, _ = self.lstm1(x)
        x = self.dropout2(x)
        x = self.batchnorm2(x)
        x, _ = self.lstm2(x)
        x = self.dropout3(x)
        x = self.batchnorm3(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.softmax(x)

        return x

In [6]:
input_size = (n_channels, samp_time)

# Random input tensor with the specified dimensions
input_tensor = torch.randn(1, *input_size)
print(input_tensor.shape)

# Forward pass through the model
#output_tensor = model(input_tensor)

net = CNN_LSTM()
print(net.forward(Variable(input_tensor)))
criterion = nn.CrossEntropyLoss() # add to device here
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-7)

torch.Size([1, 8, 750])
tensor([[0.2458, 0.7542]], grad_fn=<SoftmaxBackward0>)


In [205]:
net.forward(Variable(torch.Tensor(x_train)))

tensor([[0.5073, 0.4927],
        [0.4460, 0.5540],
        [0.5617, 0.4383],
        [0.4296, 0.5704],
        [0.3431, 0.6569],
        [0.3677, 0.6323],
        [0.6652, 0.3348],
        [0.4245, 0.5755],
        [0.1820, 0.8180],
        [0.2770, 0.7230],
        [0.6591, 0.3409],
        [0.4861, 0.5139],
        [0.3744, 0.6256],
        [0.5573, 0.4427],
        [0.7206, 0.2794],
        [0.3871, 0.6129],
        [0.4778, 0.5222],
        [0.4854, 0.5146],
        [0.4208, 0.5792],
        [0.6534, 0.3466],
        [0.5711, 0.4289],
        [0.3951, 0.6049],
        [0.3185, 0.6815],
        [0.3667, 0.6333],
        [0.5226, 0.4774],
        [0.4820, 0.5180],
        [0.3329, 0.6671],
        [0.2253, 0.7747],
        [0.3111, 0.6889],
        [0.4289, 0.5711],
        [0.4438, 0.5562],
        [0.4198, 0.5802],
        [0.2738, 0.7262],
        [0.4925, 0.5075],
        [0.7239, 0.2761],
        [0.4494, 0.5506],
        [0.3471, 0.6529],
        [0.2346, 0.7654],
        [0.6

In [ ]:
train_hist = []
val_hist = []

num_epochs = 100

for epoch_idx in tqdm(range(num_epochs)):
    net.train()
    train_count = 0
    train_correct_count = 0
    for batch_idx, (train_x, train_y) in enumerate(train_loader):
        train_x = train_x.float()
        train_y = train_y.long()
        optimizer.zero_grad()
        logits = net(train_x)
        loss = criterion(logits, train_y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_hat = torch.argmax(logits, dim=-1)
            train_correct_count += torch.sum(y_hat == train_y, axis=-1)
            train_count += train_x.size(0)

    train_acc = train_correct_count / train_count
    train_hist.append(train_acc)

    net.eval()
    val_count = 0
    val_correct_count = 0
    with torch.no_grad():
        for idx, (val_x, val_y) in enumerate(val_loader):
            val_x = val_x.float()
            val_y = val_y.long()
            logits = net(val_x).detach()
            y_hat = torch.argmax(logits, dim=-1)
            val_correct_count += torch.sum(y_hat == val_y, axis=-1)
            val_count += val_x.size(0)
    val_acc = val_correct_count / val_count
    val_hist.append(val_acc)

    print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))

### Data

In [190]:
X = np.load('preprocessed_data.npy')

In [195]:
X = np.swapaxes(X, 1,2)

In [196]:
X.shape

(69, 8, 750)

In [197]:
x_train = X

In [201]:
y_train = valence

### Labels

In [104]:
emocsv = pd.read_csv("emotions_1.csv")
emocsv2 = emocsv.iloc[:,4:]

In [130]:
new_cols = ["How feel", "Pos", "Energ", "Dom", "Content", "Amused", "Angry", "Sad", "Disgust", "Afraid", "Emo"]

In [161]:
emo_labels = pd.DataFrame()

for row in range(3):
    sel_row = emocsv2.iloc[row]
    for i in np.arange(0, emocsv2.shape[1], 11):
        obs = sel_row[i:i+11].to_frame().T
        obs = obs.rename(columns={obs.columns[i]: new_cols[i] for i in range(11)})
        emo_labels = pd.concat([emo_labels, obs])

In [170]:
emo_labels['Pos2'] = (emo_labels['Pos'] > 4) + 0
emo_labels['Energ2'] = (emo_labels['Energ'] > 4) + 0
emo_labels['Dom2'] = (emo_labels['Dom'] > 4) + 0

In [200]:
valence = emo_labels['Pos2'].to_numpy()
valence

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [199]:
emo_labels

,How feel,Pos,Energ,Dom,Content,Amused,Angry,Sad,Disgust,Afraid,Emo,Pos2,Energ2,Dom2
0,"disgusted, but the music made the video funny,...",2,3,7,1,2,4,1,5,1,Disgusted,0,0,1
0,disgusted,2,3,7,1,1,1,1,5,1,Disgusted,0,0,1
0,disgusted,1,5,7,1,1,1,1,5,1,Disgusted,0,1,1
0,relaxed,6,5,8,3,1,1,1,1,1,Neutral (unemotional),1,1,1
0,"moved, sad",3,4,3,1,1,1,3,1,1,Sad,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1


In [172]:
formal_emotions = emo_labels['Emo']

In [179]:
formal_emotions = formal_emotions.to_numpy()

In [187]:
formal_emotions = formal_emotions.reshape((formal_emotions.size, 1))

In [189]:
from sklearn.preprocessing import OneHotEncoder
cat = OneHotEncoder()
emotions_onehot = cat.fit_transform(formal_emotions).toarray()
emotions_onehot

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 